# Brandon Vasquez

### The following work was done without collaboration from other peers within the class but with aid from the SciKit library documentation page for the various methods, variables, and functions used.

## Sources used (among the documentation of the various libraries included):
* [Plot multi-class SGD on the iris dataset from SciKit Documentation](https://scikit-learn.org/stable/auto_examples/linear_model/plot_sgd_iris.html)
* [Basically what we have to do...but for the SVM classifier](https://scikit-learn.org/stable/auto_examples/svm/plot_iris_svc.html)



In [1]:
from sklearn import datasets, svm
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, Rando
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

ImportError: cannot import name 'Rando' from 'sklearn.tree' (C:\Users\Brandon\Desktop\Github repos\Intro_to_ML_assignments\venv\Lib\site-packages\sklearn\tree\__init__.py)

In [ ]:
iris_data = datasets.load_iris()
features = ["Sepal Length", "Sepal Width", "Petal Length", "Petal Width"]

match_targets = np.array([iris_data.target_names[target] for target in iris_data.target]) # match each target name to the respective target index
iris_df = pd.DataFrame(iris_data.data, columns=features, index=match_targets)

print(iris_df.head(10))
print(match_targets, len(match_targets))

In [18]:
# helper functions here....

In [20]:
sep_feats = iris_df[['Sepal Length', 'Sepal Width']]
print(type(sep_feats), sep_feats[::10], sep_feats.head(5))

<class 'pandas.core.frame.DataFrame'>             Sepal Length  Sepal Width
setosa               5.1          3.5
setosa               5.4          3.7
setosa               5.4          3.4
setosa               4.8          3.1
setosa               5.0          3.5
versicolor           7.0          3.2
versicolor           5.0          2.0
versicolor           5.9          3.2
versicolor           5.5          2.4
versicolor           5.5          2.6
virginica            6.3          3.3
virginica            6.5          3.2
virginica            6.9          3.2
virginica            7.4          2.8
virginica            6.7          3.1         Sepal Length  Sepal Width
setosa           5.1          3.5
setosa           4.9          3.0
setosa           4.7          3.2
setosa           4.6          3.1
setosa           5.0          3.6


# Discussion

## Large Delta of two feature classifications when using Perceptron loss function
* Choosing two distinct sets of features it was observed that the loss functions effect on the classifying the data drastically changed as observed in the 'Scores across the various # of features' it can be seen that when we compare the accurracy of the perceptron loss function on the two different sets of features this was where the greatest disparity was seen. When now reflecting on the plot featuring the decision regions of the respective classes we see that for the set consisting of the features 'Petal Width, Sepal Width' there was a gross misclassification for the Versicolor class where the plotted training points extend into the decision regions--not to be confused with the hyperplane--of both setosa and virginica. Now comparing this classification plot to that of the set with 'Petal Width, Petal Length' it can be observed that the higher score of the classification is due to the more accurate split of the decision regions where the training data--while still not completely linearly seperable. This delta between the two further shows the effect that linearly inseperable data has on the percetron loss function and thus the classifier as a whole.

## Accurracy Across the chosen combination of features...
* When looking across all of the features in the plot 'Scores across the various combination of features' tt is also noted that using the perceptron loss function on the three-feature combination as well as the two distinct two-feature combinations resulted in the lowest accuracy score, however, the four-feature combination exhibited the second highest accuracy score with the perceptron loss function. The highest accuracy score recorded using the four loss functions for the ‘Sepal Width, Petal Width’ two-feature and four-feature combination was using the hinge loss function meanwhile for the ‘Petal Width, Petal Length’ and the three-feature loss function the highest accuracy score was recorded with the squared hinge loss function. 

* Inclusively the 'Petal Width, Sepal Width' two-feature combination exhibited similar deltas in accurracy and even slightly converged on the loss function that resulted in the highest accuracy score for both--squared hinge--this could have possibly been due to the similarity in the distribution of the training data, when observing the 'loss function: squared_hinge with score 0.933' from the 3D plot of the three feature classification combination and the 'loss function: squared_hinge with score 0.933' in 2D featuring the 'Petal Width, Sepal Width' two-feature combination it can be observed that the distribution of the three species is quite similar with the setosa class being linearly seperable from the other two class meanwhile versicolor and virginica are not linearly seperable.







